In [2]:
import itertools

# Our numerical workhorses
import numpy as np
import pandas as pd
import scipy.integrate

# Import Altair for high level plotting
import altair as alt

# Import Bokeh modules for interactive plotting
import bokeh.io
import bokeh.plotting

# Set up Bokeh for inline viewing
bokeh.io.output_notebook()
# Allow Altair to work in notebooks

Loading BokehJS ...

In [3]:
# Use pd.read_csv() to read in the data and store in a DataFrame
df = pd.read_csv('../data/ant_joint_locations.csv', comment='#')
# Make new columns converting pixels to mm.
df['x (mm)'] = df['x_coord'] * .08
df['y (mm)'] = df['y_coord'] * .08

# Check if it worked
df.head()

,frame,beetle_treatment,ID,bodypart,x_coord,y_coord,likelihood,x (mm),y (mm)
0,0,dalotia,0,head,73.086,193.835,1.0,5.84688,15.50680
1,1,dalotia,0,head,73.730,194.385,1.0,5.89840,15.55080
2,2,dalotia,0,head,75.673,195.182,1.0,6.05384,15.61456
3,3,dalotia,0,head,77.319,196.582,1.0,6.18552,15.72656
4,4,dalotia,0,head,78.128,197.891,1.0,6.25024,15.83128


In [4]:
# Make new columns converting frames to time.
df['time'] = df['frame'] / 28

# Check if it worked
df

,frame,beetle_treatment,ID,bodypart,x_coord,y_coord,likelihood,x (mm),y (mm),time
0,0,dalotia,0,head,73.086,193.835,1.000,5.84688,15.50680,0.000000
1,1,dalotia,0,head,73.730,194.385,1.000,5.89840,15.55080,0.035714
2,2,dalotia,0,head,75.673,195.182,1.000,6.05384,15.61456,0.071429
3,3,dalotia,0,head,77.319,196.582,1.000,6.18552,15.72656,0.107143
4,4,dalotia,0,head,78.128,197.891,1.000,6.25024,15.83128,0.142857
5,5,dalotia,0,head,79.208,198.697,1.000,6.33664,15.89576,0.178571
6,6,dalotia,0,head,79.663,198.069,1.000,6.37304,15.84552,0.214286
7,7,dalotia,0,head,81.485,198.142,1.000,6.51880,15.85136,0.250000
8,8,dalotia,0,head,81.835,198.350,1.000,6.54680,15.86800,0.285714
9,9,dalotia,0,head,83.263,197.934,1.000,6.66104,15.83472,0.321429


Ideally we would like to graph the speed of the ant, since this is most likely to be indicative of activity level. However, we worry that inaccuracies in the detection of the thorax from frame to frame might make instantaneous speed data very difficult to interpret. I will try to average the position of each ant over 5 frames, which will be a little less than $\frac{1}{5}$ of a second.

In [7]:
ids = df["ID"].values
x_vals = df["x (mm)"].values
y_vals = df["y (mm)"].values
current_ant = 0 # This will keep track of the ant in question
numerator_x = 0
numerator_y = 0
denominator = 0
average_x = np.empty(len(df.index))
average_y = np.empty(len(df.index))
for (ant, x, y, i) in zip(ids, x_vals, y_vals, df.index):
    if (ant != current_ant): # If the ant changed we need to save what we have.
        current_ant = ant    
        avg_x = numerator_x / denominator
        avg_y = numerator_y / denominator
        # Save values prior to this one
        for num in range(0, denominator):
            average_x[i - 1 - num] = avg_x
            average_y[i - 1 - num] = avg_y
        # Reset numerator and denominator for new ant
        denominator = 0
        numerator_x = 0
        numerator_y = 0
                
    # End of dataframe conditions -> value is also valid for this location if 
    # the dataframe is ending. 
    elif (i == len(df.index) - 1): 
        numerator_x += x
        numerator_y += y
        denominator += 1
        avg_x = numerator_x / denominator
        avg_y = numerator_y / denominator
        # Save remaining frames
        for num in range(0, denominator):
            average_x[i - num] = avg_x
            average_y[i - num] = avg_y
            
    # Averages over 5 values in the normal case    
    elif denominator == 5:
        for num in range(0, 5):
            avg_x = numerator_x / denominator
            avg_y = numerator_y / denominator
            average_x[i - 1 - num] = avg_x
            average_y[i - 1 - num] = avg_y
        denominator = 0
        numerator_x = 0
        numerator_y = 0
    
    numerator_x += x
    numerator_y += y
    denominator += 1
df["Average x (mm)"] = average_x
df["Average y (mm)"] = average_y
df

,frame,beetle_treatment,ID,bodypart,x_coord,y_coord,likelihood,x (mm),y (mm),time,Average x (mm),Average y (mm)
0,0,dalotia,0,head,73.086,193.835,1.000,5.84688,15.50680,0.000000,6.046976,15.646000
1,1,dalotia,0,head,73.730,194.385,1.000,5.89840,15.55080,0.035714,6.046976,15.646000
2,2,dalotia,0,head,75.673,195.182,1.000,6.05384,15.61456,0.071429,6.046976,15.646000
3,3,dalotia,0,head,77.319,196.582,1.000,6.18552,15.72656,0.107143,6.046976,15.646000
4,4,dalotia,0,head,78.128,197.891,1.000,6.25024,15.83128,0.142857,6.046976,15.646000
5,5,dalotia,0,head,79.208,198.697,1.000,6.33664,15.89576,0.178571,6.487264,15.859072
6,6,dalotia,0,head,79.663,198.069,1.000,6.37304,15.84552,0.214286,6.487264,15.859072
7,7,dalotia,0,head,81.485,198.142,1.000,6.51880,15.85136,0.250000,6.487264,15.859072
8,8,dalotia,0,head,81.835,198.350,1.000,6.54680,15.86800,0.285714,6.487264,15.859072
9,9,dalotia,0,head,83.263,197.934,1.000,6.66104,15.83472,0.321429,6.487264,15.859072
